In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import numpy as np
import time
from sqlalchemy import create_engine
import pyodbc

In [ ]:
headers = {'Authorization': 'Token token=""'}
indicator_id = '460'
start_date = '2017-01-01T00:00:00Z'
end_date = '2018-12-31T24:00:00Z'
#0a3ec68a89579911bd7610e20df0f79c83f79b10275b19025aaf414136c91f9a

In [ ]:
def get(indicator: str, start_date: str, end_date: str):

    root = f'https://api.esios.ree.es/indicators/{indicator}?&start_date={start_date}&end_date={end_date}/'

    data = requests.get(root,
                        headers=headers
                       )
    print(data.status_code)
    json = data.json()
    
    return json

In [ ]:
def df(json):
    
    table = pd.DataFrame()
    
    print("Converting JSON to DataFrame...")
    for val in tqdm(json['indicator']['values']):
        single_df = pd.DataFrame({'datetime': val['datetime_utc'], 'value': val['value']}, index=[0])
        table = table.append(single_df, ignore_index=True)

    table.insert(0, 'name', json['indicator']['name'])
    table.insert(1, 'id', json['indicator']['id'])
    table.insert(4, 'update_timestamp', json['indicator']['values_updated_at'])
    
    table.rename(columns={'value': 'demand_forecast'}, inplace=True)
    print("Conversion completed")
    
    return table

In [ ]:
prueba = get(indicator_id, '2017-01-01T00:00:00Z', '2018-12-31T24:00:00Z')

In [ ]:
df = df(prueba)

In [ ]:
df

In [ ]:
speed_df = pd.read_csv("../DataOps_test/dataset_speed_optimization.csv")

In [ ]:
speed_df

In [ ]:
# Function t o a p p l y t o a l l rows i n t h e DataFrame

def function_to_apply(lat, lon):
    a = np.sin(lat/2)**2 + np.cos(lon) * np.sin(lon/2)**2
    
    return a

In [ ]:
df = speed_df.copy()

start = time.time()

list_results = []
for i in range(0, len(df)):
    r = function_to_apply(df.iloc[i]['latitude'], df.iloc[i]['longitude'])
    list_results.append(r)
    
df['distance'] = list_results

elapsed_time_fl = (time.time() - start)
print(elapsed_time_fl)

In [ ]:
df

In [ ]:
df = speed_df.copy()

start = time.time()

df['distance'] = np.sin(df['latitude']/2)**2 + np.cos(df['longitude']) * np.sin(df['longitude']/2)**2
elapsed_time_fl = (time.time() - start)

elapsed_time_fl

In [ ]:
df = pd.read_csv("../data/david_blanco_etl.csv")

In [ ]:
df.rename(columns={"datetime": "demand_datetime", "id":"indicator_id"}, inplace=True)
df.insert(0, 'id', [x+1 for x in range(len(df))])
for col in df.select_dtypes(include="object").columns.to_list():
    df[f'{col}'] = df[f'{col}'].astype('category')
df['name'] = "Prevision diaria de la demanda electrica peninsular"
df.columns

In [ ]:
df.dtypes

In [ ]:
server = "localhost"
database = "mock_db"
user = "sa"
password = "Snowdav3"

In [ ]:
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server}; SERVER=localhost; Database=mock_db; UID=sa; PWD=Snowdav3;')
cursor = conn.cursor()

In [ ]:
# Create Table

create_table = """
    CREATE TABLE demand_forecast_esios
    (
    id int PRIMARY KEY,
    name varchar(100),
    indicator_id int,
    demand_datetime varchar(100),
    demand_forecast int,
    update_timestamp varchar(100)
    )

"""
cursor.execute(create_table)
conn.commit()

In [ ]:
cols = ','.join(df.columns.to_list())
cols

In [ ]:
sql = "INSERT INTO demand_forecast_esios ([id],[name],[indicator_id], [demand_datetime], [demand_forecast], [update_timestamp]) VALUES (?,?,?,?,?,?)"
sql_cols = df.columns.to_list()
cursor.executemany(sql, df[sql_cols].values.tolist())   
conn.commit()

In [ ]:
df.dtypes

In [ ]:
cursor.execute("DELETE FROM demand_forecast_esios")
conn.commit()

In [ ]:
df.select_dtypes(include="object").columns.to_list()

In [ ]:
df.dtypes

In [ ]:
df

In [ ]:
df.columns.to_list()

In [ ]:
speed_df = pd.read_csv("../DataOps_test/dataset_speed_optimization.csv")
speed_df

In [ ]:
def function_to_apply(lat, lon):
    a = np.sin(lat / 2) ** 2 + np.cos(lon) * np.sin(lon / 2) ** 2

    return a

In [ ]:
# iloc
df = speed_df.copy()

In [ ]:
%%timeit 

list_results = []
for i in range(0, len(df)):
    r = function_to_apply(df.iloc[i]['latitude'], df.iloc[i]['longitude'])
    list_results.append(r)

df['distance'] = list_results

In [ ]:
# apply
df = speed_df.copy()

In [ ]:
def ftapply(row):
    return np.sin(row['latitude']/2)**2 + np.cos(row['longitude']) * np.sin(row['longitude']/2)**2

In [ ]:
%%timeit 

df['distance'] = df.apply(ftapply, axis=1)

In [ ]:
df

In [ ]:
%%timeit
df['distance'] = np.sin(df['latitude']/2)**2 + np.cos(df['longitude']) * np.sin(df['longitude']/2)**2

In [ ]:
df

In [ ]:
%%timeit
df['distance'] = None
df['distance'].map(lambda x: np.sin(df['latitude']/2)**2 + np.cos(df['longitude']) * np.sin(df['longitude']/2)**2)

In [39]:
df = pd.read_csv("../DataOps_test/dataset_memory_optimization.csv")

In [40]:
df

,ean_hotel_id,name,address1,city,state_province,postal_code,latitude,longitude,star_rating,high_rate,low_rate
0,269955,Hilton Garden Inn Albany/SUNY Area,1389 Washington Ave,Albany,NY,12206,42.68751,-73.81643,3.0,154.0272,124.0216
1,113431,Courtyard by Marriott Albany Thruway,1455 Washington Avenue,Albany,NY,12206,42.68971,-73.82021,3.0,179.0100,134.0000
2,108151,Radisson Hotel Albany,205 Wolf Rd,Albany,NY,12205,42.72410,-73.79822,3.0,134.1700,84.1600
3,254756,Hilton Garden Inn Albany Medical Center,62 New Scotland Ave,Albany,NY,12208,42.65157,-73.77638,3.0,308.2807,228.4597
4,198232,CrestHill Suites SUNY University Albany,1415 Washington Avenue,Albany,NY,12206,42.68873,-73.81854,3.0,169.3900,89.3900
...,...,...,...,...,...,...,...,...,...,...,...
417531,324259,Residence Inn Yonkers Westchester County,7 Executive Blvd,Yonkers,NY,10701,40.97275,-73.88075,3.0,269.3600,169.3600
417532,107949,Ramada Inn Yonkers,125 Tuckahoe Rd,Yonkers,NY,10710,40.95466,-73.86483,2.5,129.0000,119.0000
417533,509723,Hyatt Place New York/Yonkers,7000 Mall Walk,Yonkers,NY,10704,40.92625,-73.85438,3.0,249.3100,199.3100
417534,621870,Hampton Inn & Suites Yonkers - Westchester,555 Tuckahoe Rd,Yonkers,NY,10710,40.95375,-73.84935,2.5,189.1900,134.1800


In [41]:
df.dtypes

ean_hotel_id        int64
name               object
address1           object
city               object
state_province     object
postal_code        object
latitude          float64
longitude         float64
star_rating       float64
high_rate         float64
low_rate          float64
dtype: object

In [42]:
initial_memory_usage = df.memory_usage(deep=True)
initial_memory_usage

Index                  128
ean_hotel_id       3340288
name              35920128
address1          30833152
city              27546880
state_province    24634624
postal_code       25908992
latitude           3340288
longitude          3340288
star_rating        3340288
high_rate          3340288
low_rate           3340288
dtype: int64

# Object to category

In [43]:
type_object = df.select_dtypes(include='object').columns.to_list()
df[type_object].memory_usage(deep=True)

Index                  128
name              35920128
address1          30833152
city              27546880
state_province    24634624
postal_code       25908992
dtype: int64

In [44]:
for col in type_object:
    df[f'{col}'] = df[f'{col}'].astype("category")

In [45]:
df[type_object].memory_usage(deep=True)

Index                 128
name              1039412
address1          1020869
city               864326
state_province     417703
postal_code        880291
dtype: int64

# Downscaling numeric types

In [46]:
df["ean_hotel_id"].memory_usage(deep=True)

3340416

In [47]:
df["ean_hotel_id"] = pd.to_numeric(df["ean_hotel_id"], downcast="unsigned")

In [48]:
df["ean_hotel_id"].memory_usage(deep=True)

1670272

In [49]:
float_type = df.select_dtypes(include='float').columns.to_list()
df[float_type].memory_usage(deep=True)

Index              128
latitude       3340288
longitude      3340288
star_rating    3340288
high_rate      3340288
low_rate       3340288
dtype: int64

In [50]:
#for col in float_type:
df[float_type] = df[float_type].apply(pd.to_numeric, downcast="float")

In [51]:
df[float_type].memory_usage(deep=True)

Index              128
latitude       1670144
longitude      1670144
star_rating    1670144
high_rate      1670144
low_rate       1670144
dtype: int64

In [52]:
df.dtypes

ean_hotel_id        uint32
name              category
address1          category
city              category
state_province    category
postal_code       category
latitude           float32
longitude          float32
star_rating        float32
high_rate          float32
low_rate           float32
dtype: object

In [55]:
new_memory_usage = pd.DataFrame(df.memory_usage(deep=True))
new_memory_usage

,0
Index,128
ean_hotel_id,1670144
name,1039412
address1,1020869
city,864326
state_province,417703
postal_code,880291
latitude,1670144
longitude,1670144
star_rating,1670144


In [79]:
result = pd.concat([initial_memory_usage, new_memory_usage], axis=1)

In [80]:
result.columns = ["initial_memory_usage", "new_memory_usage"]

In [81]:
result['percentual_change'] = (result['new_memory_usage'] - result['initial_memory_usage']) / result['initial_memory_usage']*100

In [82]:
result['percentual_change'] = result['percentual_change'].round(decimals=2).astype(str) + '%'

In [83]:
result

,initial_memory_usage,new_memory_usage,percentual_change
Index,128,128,0.0%
ean_hotel_id,3340288,1670144,-50.0%
name,35920128,1039412,-97.11%
address1,30833152,1020869,-96.69%
city,27546880,864326,-96.86%
state_province,24634624,417703,-98.3%
postal_code,25908992,880291,-96.6%
latitude,3340288,1670144,-50.0%
longitude,3340288,1670144,-50.0%
star_rating,3340288,1670144,-50.0%
